In [2]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")
import re
import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import time
import numpy as np
from sklearn.model_selection import KFold
import statistics

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import f1_score
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

[nltk_data] Error loading vader_lexicon: <urlopen error [WinError
[nltk_data]     10060] A connection attempt failed because the
[nltk_data]     connected party did not properly respond after a
[nltk_data]     period of time, or established connection failed
[nltk_data]     because connected host has failed to respond>


In [26]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.feature_selection import SelectFromModel, VarianceThreshold

In [3]:
def data_preprocessing(path, test=False):
    #start_time = time.time()
    
    #create appropriate file path
    if test == False:
        pfilename = path + "/product_training.json"
        rfilename = path + "/review_training.json"
    else:
        pfilename = path + "/product_test.json"
        rfilename = path + "/review_test.json"
    
    #extract files as pandas dataframes
    product_df = pd.read_json(pfilename)
    
    review_df = pd.read_json(rfilename).drop_duplicates(subset=["reviewerID", "unixReviewTime"], keep="first")
    ## 11.66 seconds to get to here
    
    review_df.drop(columns=["reviewerID","vote", "unixReviewTime","reviewTime","style","reviewerName","image"], axis=1 ,inplace=True)
    
    review_df['reviewText'].fillna("", inplace=True)
    review_df['summary'].fillna("", inplace=True)
    
    review_df.sort_values('asin', inplace = True)
    product_df.sort_values('asin', inplace = True)
    
    group = review_df.groupby("asin")
    
    #review_group_df = pd.DataFrame(columns = ['asin', 'numReviews', 'percentVerified','reviewText','summaryText', 'awesomeness'])
    
    # about the same amount of time to get to here
    start_time = time.time()
    datalist = []
    count = 0
    #awesome_pos = 0
    for asin, data in group:
        verifiedCount = data['verified'].sum()
        reviewCount = data['asin'].count()
        percentVerified = verifiedCount / reviewCount
        if count == 0:
            print(type(data['reviewText']))
        reviewText = ' '.join(data['reviewText'])
        #reviewText = ' '.join(transform_document(x) for x in data['reviewText'])
        #summaryText = ""
        summaryText = ' '.join(data['summary'])
        #summaryText = ' '.join(transform_document(x) for x in data['summary'])
        #reviewText = transform_document(' '.join(data['reviewText']))
        #summaryText = transform_document(' '.join(data['summary']))
        #awesomeness = 0
        
        #SENTIMENT ANALYSIS CHUNK
        (rev_means, rev_stdevs) = sentiment_analysis(data['reviewText'])
        (sum_means, sum_stdevs) = sentiment_analysis(data['summary'])
        while (product_df['asin'][count] != asin):
               count = count + 1
        
        awesomeness = product_df['awesomeness'][count]
        #awesome_pos = awesome_pos + reviewCount
        #awesomeness = product_df.loc[product_df['asin'] == asin, 'awesomeness'].values[0] #might be slow
        datalist.append([asin,  reviewCount, percentVerified, reviewText, \
                         summaryText, rev_means[0], rev_means[1], rev_means[2], \
                         rev_stdevs[0], rev_stdevs[1], rev_stdevs[2], sum_means[0], \
                         sum_means[1], sum_means[2], sum_stdevs[0], sum_stdevs[1], \
                         sum_stdevs[2], awesomeness])
        
        count = count + 1
        #if count > 100:
        #    break
        
        '''new_row = {'asin': asin, 
                   'numReviews': reviewCount, 
                   'percentVerified': percentVerified, 
                   'reviewText': transform_document(' '.join(data['reviewText'])), 
                   'summaryText': transform_document(' '.join(data['summary'])), 
                   'awesomeness': product_df.loc[product_df['asin'] == asin, 'awesomeness'].values[0]} 
        review_group_df = review_group_df.append(new_row, ignore_index = True)
         '''
    review_group_df = pd.DataFrame(datalist,columns =['asin', 'numReviews', 'percentVerified','reviewText','summaryText', \
                                                      'reviewPosMean', 'reviewNeuMean', 'reviewNegMean', 'reviewPosStDev', \
                                                      'reviewNeuStDev', 'reviewNegStDev', 'summaryPosMean', 'summaryNeuMean', \
                                                      'summaryNegMean', 'summaryPosStDev','summaryNeuStDev','summaryNegStDev', \
                                                      'awesomeness'])    
    
    review_group_df.to_json(r'../devided_dataset_v2/CDs_and_Vinyl/train/cleaned_data_new.json')
    end_time = time.time()
    print(end_time - start_time)
    
    return review_group_df
    

In [5]:
# Preprocessed Data Generated (Reviews and Summaries aggregated, no NLP processing)
#review_group_df = data_preprocessing("../devided_dataset_v2/CDs_and_Vinyl/train")
#review_group_df = pd.read_json('../devided_dataset_v2/CDs_and_Vinyl/train/cleaned_data.json')
review_group_df = pd.read_json('cleaned_data_new.json')
#review_group_df.to_json("preprocessed.json")
#review_group_df.head()
#review_group_df

In [ ]:
def data_preprocessing_fast(path, test=False):
    #start_time = time.time()
    
    #create appropriate file path
    if test == False:
        pfilename = path + "/product_training.json"
        rfilename = path + "/review_training.json"
    else:
        pfilename = path + "/product_test.json"
        rfilename = path + "/review_test.json"
    
    #extract files as pandas dataframes
    product_df = pd.read_json(pfilename)
    
    review_df = pd.read_json(rfilename).drop_duplicates(subset=["reviewerID", "unixReviewTime"], keep="first")
    ## 11.66 seconds to get to here
    
    review_df.drop(columns=["reviewerID","vote", "unixReviewTime","reviewTime","style","reviewerName","image"], axis=1 ,inplace=True)
    
    review_df['reviewText'].fillna("", inplace=True)
    review_df['summary'].fillna("", inplace=True)
    
    review_df.sort_values('asin', inplace = True)
    product_df.sort_values('asin', inplace = True)
    
    group = review_df.groupby("asin")
    
    #review_group_df = pd.DataFrame(columns = ['asin', 'numReviews', 'percentVerified','reviewText','summaryText', 'awesomeness'])
    
    # about the same amount of time to get to here
    start_time = time.time()
    datalist = []
    count = 0
    #awesome_pos = 0
    for asin, data in group:
        verifiedCount = data['verified'].sum()
        reviewCount = data['asin'].count()
        percentVerified = verifiedCount / reviewCount
        if count == 0:
            print(type(data['reviewText']))
        #reviewText = ' '.join(data['reviewText'])
        #reviewText = ' '.join(transform_document(x) for x in data['reviewText'])
        #summaryText = ""
        #summaryText = ' '.join(data['summary'])
        #summaryText = ' '.join(transform_document(x) for x in data['summary'])
        #reviewText = transform_document(' '.join(data['reviewText']))
        #summaryText = transform_document(' '.join(data['summary']))
        #awesomeness = 0
        
        #SENTIMENT ANALYSIS CHUNK
        (rev_means, rev_stdevs) = sentiment_analysis(data['reviewText'])
        (sum_means, sum_stdevs) = sentiment_analysis(data['summary'])
        while (product_df['asin'][count] != asin):
               count = count + 1
        
        awesomeness = product_df['awesomeness'][count]
        #awesome_pos = awesome_pos + reviewCount
        #awesomeness = product_df.loc[product_df['asin'] == asin, 'awesomeness'].values[0] #might be slow
        datalist.append([asin,  reviewCount, percentVerified, \
                         rev_means[0], rev_means[1], rev_means[2], \
                         rev_stdevs[0], rev_stdevs[1], rev_stdevs[2], sum_means[0], \
                         sum_means[1], sum_means[2], sum_stdevs[0], sum_stdevs[1], \
                         sum_stdevs[2], awesomeness])
        
        count = count + 1
        #if count > 100:
        #    break
        
        '''new_row = {'asin': asin, 
                   'numReviews': reviewCount, 
                   'percentVerified': percentVerified, 
                   'reviewText': transform_document(' '.join(data['reviewText'])), 
                   'summaryText': transform_document(' '.join(data['summary'])), 
                   'awesomeness': product_df.loc[product_df['asin'] == asin, 'awesomeness'].values[0]} 
        review_group_df = review_group_df.append(new_row, ignore_index = True)
         '''
    review_group_df = pd.DataFrame(datalist,columns =['asin', 'numReviews', 'percentVerified','reviewText','summaryText', \
                                                      'reviewPosMean', 'reviewNeuMean', 'reviewNegMean', 'reviewPosStDev', \
                                                      'reviewNeuStDev', 'reviewNegStDev', 'summaryPosMean', 'summaryNeuMean', \
                                                      'summaryNegMean', 'summaryPosStDev','summaryNeuStDev','summaryNegStDev', \
                                                      'awesomeness'])    
    
    review_group_df.to_json(r'../devided_dataset_v2/CDs_and_Vinyl/train/cleaned_data_new.json')
    end_time = time.time()
    print(end_time - start_time)
    
    return review_group_df

In [5]:
def sentiment_analysis(docs):
    sentiments = []
    sid = SentimentIntensityAnalyzer()
    for doc in docs:
        polarities = sid.polarity_scores(doc)
        sentiments.append((polarities['pos'], polarities['neu'], polarities['neg']))
    if len(sentiments) == 1:
        return (sentiments[0], (0, 0, 0))
    else:
        pos = [sents[0] for sents in sentiments]
        neu = [sents[1] for sents in sentiments]
        neg = [sents[2] for sents in sentiments]
        return ((statistics.mean(pos), statistics.mean(neu), statistics.mean(neg)), (statistics.stdev(pos), statistics.stdev(neu), statistics.stdev(neg)))
    

In [6]:
string_transformer = Pipeline(
    steps = [('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer())]
)
wordbagger = ColumnTransformer(
    transformers=[("rev", string_transformer, 'reviewText'), 
                  ("sum", string_transformer, 'summaryText')]
    , remainder='passthrough'
)

clf = Pipeline(steps = [("wordbag", wordbagger), 
                        ("scale", MaxAbsScaler()), 
                        ('classifier', LogisticRegression(solver="newton-cg", C=0.05, max_iter = 500, n_jobs=-1))])
#x = review_group_df.filter(['numReviews', 'percentVerified', 'reviewText', 'summaryText'])
#clf = LogisticRegression(tol = 0.001, max_iter = 150)
#clf.fit(x,y)
review_features = review_group_df.filter(['numReviews', 'percentVerified', 'reviewText', 'summaryText', \
                                          'reviewPosMean', 'reviewNeuMean', 'reviewNegMean', 'reviewPosStDev', \
                                          'reviewNeuStDev', 'reviewNegStDev', 'summaryPosMean', 'summaryNeuMean', \
                                          'summaryNegMean', 'summaryPosStDev','summaryNeuStDev','summaryNegStDev'])
fewer_features = review_group_df.filter(['numReviews', 'percentVerified', \
                                          'reviewPosMean', 'reviewNeuMean', 'reviewNegMean', 'reviewPosStDev', \
                                          'reviewNeuStDev', 'reviewNegStDev', 'summaryPosMean', 'summaryNeuMean', \
                                          'summaryNegMean', 'summaryPosStDev','summaryNeuStDev','summaryNegStDev'])
y = review_group_df.filter(['awesomeness'])

In [8]:
preprocessor = Pipeline(steps = [("wordbag", wordbagger), 
                                  ("scale", MaxAbsScaler())])
processed_features = preprocessor.fit_transform(review_features)
smaller_processed_features = MaxAbsScaler().fit_transform(fewer_features)

In [113]:
select_clf = SelectFromModel(threshold = "1.5*mean", \
                             estimator=LogisticRegression(solver="newton-cg", C=0.05, max_iter = 500, n_jobs=-1))
log_clf = LogisticRegression(solver="newton-cg", C=0.05, max_iter = 500, n_jobs=-1)
multi_clf = MultinomialNB()
random_forest_clf = RandomForestClassifier(max_depth = 150, n_jobs = -1)
boosted_clf = GradientBoostingClassifier()
#just_clf.fit(processed_features, y)

voting_clf = VotingClassifier(estimators=[('RF', random_forest_clf), ('mN', multi_clf), ('LR', log_clf)], voting='hard')

In [116]:
var_processed_features = VarianceThreshold().fit_transform(processed_features)
more_processed_features = select_clf.fit_transform(var_processed_features, np.ravel(y))
start = time.time()
print("I started at " + str(start))
# this runs the k-fold cross-validation automatically?
cv10_results = cross_validate(log_clf, more_processed_features, np.ravel(y), cv=10, n_jobs = -1, scoring = ['f1_macro', 'precision', 'recall'])
end = time.time()
print((end - start)/60)

I started at 1683787430.7084897
28.527466130256652


In [115]:
#print(cv10_results)
#print(cv10_results)
print(statistics.mean(cv10_results['test_f1_macro']))
#print(statistics.stdev(cv10_results['test_f1_macro']))
print(statistics.mean(cv10_results['test_precision']))
#print(statistics.stdev(cv10_results['test_precision']))
print(statistics.mean(cv10_results['test_recall']))
#print(statistics.stdev(cv10_results['test_recall']))

0.6215531070176024
0.6304416111410547
0.7316062637253422


In [81]:
processed_features

<71543x366180 sparse matrix of type '<class 'numpy.float64'>'
	with 28605864 stored elements in Compressed Sparse Row format>

In [82]:
var_processed_features

<71543x366180 sparse matrix of type '<class 'numpy.float64'>'
	with 28605864 stored elements in Compressed Sparse Row format>

In [111]:
more_processed_features

<71543x69395 sparse matrix of type '<class 'numpy.float64'>'
	with 21945286 stored elements in Compressed Sparse Row format>

In [24]:
kf = KFold(n_splits = 10, shuffle = True)
for i, (train_index, test_index) in enumerate(kf.split(review_features)):
    start = time.time()
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    x_train = review_features.loc[train_index, :]
    x_test = review_features.loc[test_index, :]
    y_train = np.ravel(y.loc[train_index, :])
    y_test = np.ravel(y.loc[test_index,:])
    
    x_train_proc = preprocessor.fit_transform(x_train)
    x_test_proc = preprocessor.fit_transform(x_test)
    
    #x_train_features = wordbagger.fit_transform(x_train)
    #x_test_features = wordbagger.fit_transform(x_test)
    #scaler = preprocessing.MaxAbsScaler().fit(x_train_features)
    #x_train_scaled = scaler.transform(x_train_features)
    #print(type(y_train))
    #print(x_train.shape)
    #print(y_train.shape)
    #clf = Pipeline(steps = [("preprocess", preprocessor), ('classifier', LogisticRegression()) ])
    x_train_selected = select_clf.fit_transform(x_train_proc, y_train)
    x_test_selected = select_clf.fit_transform(x_test_proc, y_test)
    log_clf.fit(x_train_selected, y_train)
    
    y_pred = log_clf.predict(x_test_proc)
    #print(x_train)
    #print(y_train.shape)
    #X_trans = preprocessor.fit_transform(x_train)
    #print(x_train.shape)
    print("model score: %.3f" % f1_score(y_test, y_pred, 'macro'))
    end = time.time()
    print(end - start)

Fold 0:
  Train: index=[    0     1     2 ... 71540 71541 71542]
  Test:  index=[    4    20    22 ... 71508 71520 71523]


ValueError: X has 109762 features per sample; expecting 115607

In [16]:
solvers = ['newton-cg']
jobs = [-1]
maxiter = [100, 500, 1000]
c = [5, 2, 1, 0.5, 0.2, 0.1, 0.05, 0.02, 0.01]
param_grid={'classifier__C':c, 'classifier__solver':solvers, 'classifier__n_jobs':jobs, 'classifier__max_iter':maxiter}
grid_clf = GridSearchCV(clf, param_grid, n_jobs=-1)
start = time.time()
print("I'm starting!")
grid_clf.fit(review_features, np.ravel(y))
end = time.time()
print((end - start)/60)

I'm starting!


/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


63.023726880550385


In [60]:
review_group_df.to_json(r'../devided_dataset_v2/CDs_and_Vinyl/train/cleaned_data.json')

In [17]:
pd.DataFrame.from_dict(grid_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_classifier__max_iter,param_classifier__n_jobs,param_classifier__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,265.902761,3.981755,22.498697,0.715377,5,100,-1,newton-cg,"{'classifier__C': 5, 'classifier__max_iter': 1...",0.579356,0.581173,0.590817,0.586735,0.590369,0.585690,0.004686,25
1,263.848393,8.369911,22.481720,1.092433,5,500,-1,newton-cg,"{'classifier__C': 5, 'classifier__max_iter': 5...",0.579356,0.581173,0.590817,0.586735,0.590369,0.585690,0.004686,25
2,258.521113,0.950623,21.237789,0.196200,5,1000,-1,newton-cg,"{'classifier__C': 5, 'classifier__max_iter': 1...",0.579356,0.581173,0.590817,0.586735,0.590369,0.585690,0.004686,25
3,242.788301,6.899337,26.501668,2.905537,2,100,-1,newton-cg,"{'classifier__C': 2, 'classifier__max_iter': 1...",0.587253,0.587882,0.597666,0.594353,0.597638,0.592958,0.004568,22
4,245.064487,12.992323,22.550827,1.717254,2,500,-1,newton-cg,"{'classifier__C': 2, 'classifier__max_iter': 5...",0.587253,0.587882,0.597666,0.594353,0.597638,0.592958,0.004568,22
5,230.603351,2.580260,22.640238,0.726609,2,1000,-1,newton-cg,"{'classifier__C': 2, 'classifier__max_iter': 1...",0.587253,0.587882,0.597666,0.594353,0.597638,0.592958,0.004568,22
6,217.788828,4.732246,23.282278,1.899143,1,100,-1,newton-cg,"{'classifier__C': 1, 'classifier__max_iter': 1...",0.595220,0.596478,0.602767,0.602041,0.604697,0.600241,0.003711,19
7,219.208971,2.917052,21.258987,0.611024,1,500,-1,newton-cg,"{'classifier__C': 1, 'classifier__max_iter': 5...",0.595220,0.596478,0.602767,0.602041,0.604697,0.600241,0.003711,19
8,214.697790,8.029622,23.524169,2.573233,1,1000,-1,newton-cg,"{'classifier__C': 1, 'classifier__max_iter': 1...",0.595220,0.596478,0.602767,0.602041,0.604697,0.600241,0.003711,19
9,202.574223,15.012335,22.778429,3.025260,0.5,100,-1,newton-cg,"{'classifier__C': 0.5, 'classifier__max_iter':...",0.603816,0.608288,0.612831,0.613363,0.613573,0.610374,0.003810,16


All the older stuff is below here:

NLP Functions

In [7]:
corpus = ["I am being handed a list of documents", "Each of these documents has several unique words", "The words will represent the class of each review", "I am also removing stopwords in order to make this make more sense"]
cleaned_corpus = [transform_document(doc) for doc in corpus]
vocabulary = vocabulary_from_corpus(cleaned_corpus, True)
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)), 
                 ('tfid', TfidfTransformer())]).fit(cleaned_corpus)),

In [12]:
def get_stopwords():
    file = open('en.txt')
    stopwords = []
    for line in file:
        stopwords.append(line.rstrip())
    return stopwords

In [23]:
def transform_corpus(review_group):
    stopwords = get_stopwords()
    reviewTextSet = review_group['reviewText']
    for index in review_group.index:
        curr_parsed = nlp(reviewTextSet[index].lower())
        doclist = []
        for token in curr_parsed:
            lemma = token.lemma_
            if not(re.match("[a-z0-9]+", lemma)):
                continue
            if lemma not in stopwords:
                doclist.append(lemma)
        reviewTextSet[index] = " ".join(doclist)

In [197]:
def transform_document(doc, remove_stopwords = True):
    #new_doc = ""
    stopwords = get_stopwords() # is this slow?
    parsed_text = nlp(doc) # is this slow
    doclist = []
    for token in parsed_text:
        lemma = token.lemma_.lower()
        if re.match("[a-z0-9]+", lemma) and (remove_stopwords == False or lemma not in stopwords):
            doclist.append(lemma) # this is less slow?
    return " ".join(doclist)

In [50]:
def bag_of_words(review_text, remove_stopwords = True):
    word_bag = {}
    stopwords = get_stopwords()
    parsed_text = nlp(review_text)
    for token in parsed_text:
        lemma = token.lemma_.lower()
        if re.match("[a-z0-9]+", lemma) and (remove_stopwords == False or lemma not in stopwords):
            if lemma in word_bag:
                word_bag[lemma] += 1
            else:
                word_bag[lemma] = 1
    return word_bag

In [51]:
def vocabulary_from_corpus(corpus, remove_stopwords = True):
    vocab_set = set()
    for document in corpus:
        word_bag = bag_of_words(document, remove_stopwords)
        for word in word_bag.keys():
            vocab_set.add(word)
    return list(vocab_set)
vocabulary_from_corpus(['this is the first document', 'this document is the second document', 'and this is the third one', 'is this the first document'], False)

['first', 'the', 'second', 'third', 'be', 'and', 'this', 'one', 'document']

In [88]:
product_df = pd.read_json('../devided_dataset_v2/CDs_and_Vinyl/train/product_training.json')
review_df = pd.read_json('../devided_dataset_v2/CDs_and_Vinyl/train/review_training.json')
#len(product_df.index)

In [6]:
teststring = "First, you need to preprocess the raw text data. This may involve tasks like tokenizing the text (i.e., splitting it into individual words), removing stopwords, stemming or lemmatizing the words, and converting the text into a numerical format that can be used as input for the model. Then, you need to split the data into training and testing sets. The training set will be used to train the model, while the testing set will be used to evaluate its performance."
#teststring.lower()
parsed = nlp(teststring.lower())